# Image Classification Using IceptionV3,ResNet,VGG16

Data link : https://www.kaggle.com/puneet6060/intel-image-classification

# Importing The Required Libraries

In [1]:
import numpy as np 
import pandas as pd
import glob
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import Model,layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16

# Importing The Dataset

In [2]:
glob.glob('seg_train/*')

['seg_train\\buildings',
 'seg_train\\forest',
 'seg_train\\glacier',
 'seg_train\\mountain',
 'seg_train\\sea',
 'seg_train\\street']

In [3]:
def prepare_dataset(path, label):
    x_train = []
    y_train = []
    all_images_path = glob.glob(path+'/*.jpg')
    for image_path in all_images_path:
        img = load_img(image_path, target_size=(150,150))
        img = img_to_array(img)
        img = img/255
        x_train.append(img)
        y_train.append(label)
    return np.array(x_train), np.array(y_train)

In [4]:
paths=glob.glob('seg_train/*')
l=len('seg_train/')
labels=[]
for path in paths:
    labels.append(path[l:])
print(labels)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


# Preparing the Train and Test Set

In [5]:
x_train_building, y_train_building  = prepare_dataset("seg_train/buildings/",0)
x_train_forest, y_train_forest  = prepare_dataset("seg_train/forest/",1)
x_train_glacier, y_train_glacier  = prepare_dataset("seg_train/glacier/",2)
x_train_mount, y_train_mount  = prepare_dataset("seg_train/mountain/",3)
x_train_sea, y_train_sea  = prepare_dataset("seg_train/sea/",4)
x_train_street, y_train_street  = prepare_dataset("seg_train/street/",5)

print('building shape training data: ', x_train_building.shape, y_train_building.shape) 
print('forest training data: ', x_train_forest.shape ,y_train_forest.shape)
print('glacier training data: ', x_train_glacier.shape,y_train_glacier.shape)
print('mountain training data: ', x_train_mount.shape, y_train_mount.shape)
print('sea training data: ', x_train_sea.shape, y_train_sea.shape)
print('street training data: ', x_train_street.shape ,y_train_street.shape)

building shape training data:  (2191, 150, 150, 3) (2191,)
forest training data:  (2271, 150, 150, 3) (2271,)
glacier training data:  (2404, 150, 150, 3) (2404,)
mountain training data:  (2512, 150, 150, 3) (2512,)
sea training data:  (2274, 150, 150, 3) (2274,)
street training data:  (2382, 150, 150, 3) (2382,)


In [6]:
x_train=np.concatenate((x_train_building,x_train_forest,x_train_glacier,x_train_mount,x_train_sea,x_train_street),axis=0)
y_train=np.concatenate((y_train_building,y_train_forest,y_train_glacier,y_train_mount,y_train_sea,y_train_street),axis=0)
print(x_train.shape)
print(y_train.shape)

(14034, 150, 150, 3)
(14034,)


In [7]:
x_test_building, y_test_building  = prepare_dataset("seg_test/buildings/",0)
x_test_forest, y_test_forest = prepare_dataset("seg_test/forest/",1)
x_test_glacier, y_test_glacier = prepare_dataset("seg_test/glacier/",2)
x_test_mount, y_test_mount = prepare_dataset("seg_test/mountain/",3)
x_test_sea, y_test_sea = prepare_dataset("seg_test/sea/",4)
x_test_street, y_test_street = prepare_dataset("seg_test/street/",5)

print('building shape test data: ', x_test_building.shape, y_test_building.shape) 
print('forest test data: ', x_test_forest.shape ,y_test_forest.shape)
print('glacier test data: ', x_test_glacier.shape,y_test_glacier.shape)
print('mountain test data: ', x_test_mount.shape, y_test_mount.shape)
print('sea test data: ', x_test_sea.shape, y_test_sea.shape)
print('street test data: ', x_test_street.shape ,y_test_street.shape)

building shape test data:  (437, 150, 150, 3) (437,)
forest test data:  (474, 150, 150, 3) (474,)
glacier test data:  (553, 150, 150, 3) (553,)
mountain test data:  (525, 150, 150, 3) (525,)
sea test data:  (510, 150, 150, 3) (510,)
street test data:  (501, 150, 150, 3) (501,)


In [8]:
x_test=np.concatenate((x_test_building,x_test_forest,x_test_glacier,x_test_mount,x_test_sea,x_test_street),axis=0)
y_test=np.concatenate((y_test_building,y_test_forest,y_test_glacier,y_test_mount,y_test_sea,y_test_street),axis=0)
print(x_test.shape)
print(y_test.shape)

(3000, 150, 150, 3)
(3000,)


# Implementing Inception V3

In [9]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = 'imagenet')


In [10]:
for layer in pre_trained_model.layers:
    layer.trainable = False
    
last_layer = pre_trained_model.output
x = layers.Flatten()(last_layer)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)

model = Model(pre_trained_model.input, x) 

In [11]:
model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

In [12]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
439/439 [==============================] - 607s 1s/step - loss: 2.1431 - acc: 0.8378 - val_loss: 0.3235 - val_acc: 0.8717
Epoch 2/5
439/439 [==============================] - 604s 1s/step - loss: 0.2924 - acc: 0.8948 - val_loss: 0.3399 - val_acc: 0.8853
Epoch 3/5
439/439 [==============================] - 612s 1s/step - loss: 0.2526 - acc: 0.9089 - val_loss: 0.2983 - val_acc: 0.8977
Epoch 4/5
439/439 [==============================] - 610s 1s/step - loss: 0.2165 - acc: 0.9193 - val_loss: 0.3347 - val_acc: 0.8863
Epoch 5/5
439/439 [==============================] - 616s 1s/step - loss: 0.2061 - acc: 0.9270 - val_loss: 0.3455 - val_acc: 0.8867


# Implementing ResNet 50 (using ImageNet)

In [22]:
pretrained_model=ResNet50( input_shape=(150,150,3),
                                  include_top=False,
                                  weights='imagenet')

In [26]:
for layer in pretrained_model.layers:
     layer.trainable = False
        
last_layer = pretrained_model.get_layer('conv5_block3_out')
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)

model_resnet = Model(pretrained_model.input, x) 

In [27]:
model_resnet.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

In [28]:
model_resnet.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
439/439 [==============================] - 1026s 2s/step - loss: 1.5917 - acc: 0.4615 - val_loss: 1.1055 - val_acc: 0.5217
Epoch 2/5
439/439 [==============================] - 1029s 2s/step - loss: 1.0323 - acc: 0.5809 - val_loss: 0.9190 - val_acc: 0.6357
Epoch 3/5
439/439 [==============================] - 1030s 2s/step - loss: 0.9751 - acc: 0.6065 - val_loss: 0.9052 - val_acc: 0.6403
Epoch 4/5
439/439 [==============================] - 1080s 2s/step - loss: 0.9382 - acc: 0.6202 - val_loss: 0.8680 - val_acc: 0.6697
Epoch 5/5
439/439 [==============================] - 1073s 2s/step - loss: 0.9316 - acc: 0.6256 - val_loss: 0.9091 - val_acc: 0.6387


# Implementing VGG 16 

In [9]:
pre_trained_model = VGG16(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = 'imagenet')

58892288/58889256 [==============================] - 193s 3us/step


In [12]:
for layer in pre_trained_model.layers:
    layer.trainable = False
    
last_layer = pre_trained_model.output
x = layers.Flatten()(last_layer)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)

vgg_model = Model(pre_trained_model.input, x) 

In [13]:
vgg_model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

In [14]:
vgg_model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
439/439 [==============================] - 2556s 6s/step - loss: 0.5352 - acc: 0.8224 - val_loss: 0.4000 - val_acc: 0.8487
Epoch 2/5
439/439 [==============================] - 2499s 6s/step - loss: 0.3182 - acc: 0.8841 - val_loss: 0.4047 - val_acc: 0.8397
Epoch 3/5
439/439 [==============================] - 2529s 6s/step - loss: 0.2597 - acc: 0.9032 - val_loss: 0.3509 - val_acc: 0.8697
Epoch 4/5
439/439 [==============================] - 2688s 6s/step - loss: 0.2180 - acc: 0.9181 - val_loss: 0.3441 - val_acc: 0.8803
Epoch 5/5
439/439 [==============================] - 3303s 8s/step - loss: 0.1836 - acc: 0.9302 - val_loss: 0.3549 - val_acc: 0.8820
